In [26]:
from astropy.coordinates import EarthLocation, SkyCoord, get_sun, AltAz 
from astropy import time
m1 = SkyCoord.from_name('M1')

#dir() to see attributes

In [27]:
print(m1.frame)

<ICRS Coordinate: (ra, dec) in deg
    (83.63308333, 22.0145)>


In [40]:
sun = get_sun(time.Time.now()).transform_to(AltAz(location=EarthLocation(53.3845258962902, 6.23475766593151)))

In [41]:
print(sun)

<SkyCoord (AltAz: obstime=2021-07-03 11:55:19.431283, location=(3781835.25203023, 5089373.33486723, 688071.22792422) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt, distance) in (deg, deg, m)
    (294.33058188, 38.0766797, 1.52094142e+11)>


In [39]:
sun.alt.deg

39.023916642671054

In [22]:
import sqlite3
import datetime as dt
con = sqlite3.connect('Database.db')

In [23]:
cur = con.cursor()

In [3]:
cur.execute("""CREATE TABLE Schedule
            (object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date)""")

In [18]:

cur.execute("""INSERT INTO Schedule VALUES
                ('{}', '{}', '{}', '{}')""".format(dt.datetime.now()))

In [11]:
cur.execute("""DROP TABLE observatory_status""")

In [24]:
def tables_in_sqlite_db(conn):
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cursor.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cursor.close()
    return tables
tables_in_sqlite_db(con)

['observatory_status', 'Schedule']

In [2]:
from Helper_funcs import sqlite_add_to_table
import sqlite3
connect = sqlite3.connect('Database.db')
sqlite_add_to_table(connect, 'Schedule', ['M1', 'Yes','moderator', 2, 3600, 2012])

INSERT INTO Schedule(object, time_sensitive, Observer_type, Rarity, total_length, Submission_Date) VALUES('M1', 'Yes', 'moderator', '2', '3600', '2012');


0

In [31]:
#Equiv to .schema
string = ''
for item in cur.execute('''PRAGMA table_info(observatory_status);'''):
    string += item[1]+', '
print(string)
for row in cur.execute('SELECT * FROM observatory_status'):
        print(row)

date, dome_status, dome_direction, telescope_direction, 
('2021-07-03 15:48:44.347587', 'closed', '[30, 40]', '[50, 60]')


In [5]:
from Helper_funcs import sqlite_get_columns
import sqlite3
connect = sqlite3.connect('Database.db')
{key: None for key in sqlite_get_columns(connect, 'observatory_status').strip('()').split(', ')}

{'date': None,
 'dome_status': None,
 'dome_direction': None,
 'telescope_direction': None}

In [59]:
connect = sqlite3.connect('Database.db')
with connect:
    for item in connect.execute('''PRAGMA table_info({});'''.format('Schedule')).fetchall():
        print(item[1])
    for row in connect.execute('SELECT * FROM {}'.format('Schedule')).fetchall():
        print(row)

object
time_sensitive
Observer_type
Rarity
total_length
Submission_Date
